In [2]:
#|default_exp showdoc

# showdoc
> Display symbol documentation in notebook and website

In [596]:
#|export
from fastcore.docments import *
from fastcore.utils import *
from importlib import import_module
from nbprocess.doclinks import *
import inspect
from collections import OrderedDict

from nbprocess.read import get_config

## Rendering docment Tables - 

In [598]:
#|export
def _non_empty_keys(d:dict):
    return [k for k,v in d.items() if v not in [inspect._empty, None] and v]

In [552]:
_test = { 'anno': inspect._empty,
          'default': bool,
          'docment': None}
test_eq(_non_empty_keys(_test), ['default'])

In [553]:
#|export
def _maybe_nm(o): 
    if o in [inspect._empty, None]: return ''
    else: return o.__name__ if hasattr(o, '__name__') else str(o)

In [554]:
test_eq(_maybe_nm(list), 'list')
test_eq(_maybe_nm('fastai'), 'fastai')

In [555]:
#|export
def _list2row(l:list): 
    return '| '+' | '.join([_maybe_nm(o) for o in l]) + ' |'

In [556]:
#|hide
test_eq(_list2row(['Hamel', 'Jeremy']), '| Hamel | Jeremy |')
test_eq(_list2row([inspect._empty, bool, 'foo']), '|  | bool | foo |')

In [599]:
#|export
class DocmentTbl:
    # this is the column order we want these items to appear
    _map = OrderedDict({'anno':'Type', 'default':'Default', 'docment':'Details'})
    
    def __init__(self, obj, verbose=False, returns=True):
        "Compute the docment table string"
        self.verbose=verbose
        self.params = L(inspect.signature(obj).parameters.keys())
        self.dm = docments(obj, full=True, returns=returns)
    
    @property
    def _columns(self):
        "Compute the set of fields that have at least one non-empty value"
        cols = set(flatten(L(self.dm.values()).filter().map(_non_empty_keys)))
        candidates = self._map if self.verbose else {'docment': 'Details'}
        return {k:v for k,v in candidates.items() if k in cols}
    
    @property
    def has_docment(self): return bool(self._columns)
    
    def _row(self, nm, props): 
        "unpack data for single row to correspond with column names."
        return [nm] + [props[c] for c in self._columns]
    
    @property
    def _row_list(self):
        "unpack data for all rows."
        ordered_params = [(p, self.dm[p]) for p in self.params]
        return L([self._row(nm, props) for nm,props in ordered_params])
    
    @property
    def _hdr_list(self): return ['  '] + [f'**{l}**' for l in L(self._columns.values())]

    @property
    def hdr_str(self):
        "The markdown string for the header portion of the table"
        md = _list2row(self._hdr_list)
        return md + '\n' + _list2row(['-' * len(l) for l in self._hdr_list])
    
    @property
    def params_str(self): 
        "The markdown string for the parameters portion of the table."
        return '\n'.join(self._row_list.map(_list2row))
    
    @property
    def table_str(self): 
        return '\n'.join([self.hdr_str, self.params_str]) if self._columns else ''

In [600]:
def foo(a, 
        b, # a docment 
        c:str): ...

In [601]:
print(DocmentTbl(foo, verbose=True).table_str)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| a |  |  |
| b |  | a docment |
| c | str |  |


# Hamel Experiment End -  

In [4]:
#|export
def get_name(obj):
    "Get the name of `obj`"
    if hasattr(obj, '__name__'):       return obj.__name__
    elif getattr(obj, '_name', False): return obj._name
    elif hasattr(obj,'__origin__'):    return str(obj.__origin__).split('.')[-1] #for types
    elif type(obj)==property:          return _get_property_name(obj)
    else:                              return str(obj).split('.')[-1]

In [5]:
#|export
def qual_name(obj):
    "Get the qualified name of `obj`"
    if hasattr(obj,'__qualname__'): return obj.__qualname__
    if inspect.ismethod(obj):       return f"{get_name(obj.__self__)}.{get_name(fn)}"
    return get_name(obj)

In [6]:
#|export
class ShowDocRenderer:
    def __init__(self, sym, disp:bool=True):
        "Show documentation for `sym`"
        store_attr()
        self.nm = qual_name(sym)
        self.isfunc = inspect.isfunction(sym)
        self.sig = inspect.signature(sym)
        self.docs = docstring(sym)

In [7]:
#|export
class BasicMarkdownRenderer(ShowDocRenderer):
    def _repr_markdown_(self):
        doc = '---\n\n'
        if self.isfunc: doc += '#'
        doc += f'### {self.nm}\n\n> **`{self.nm}`**` {self.sig}`'
        if self.docs: doc += f"\n\n{self.docs}"
        return doc

In [8]:
#|export
def show_doc(sym, disp=True, renderer=None):
    if renderer is None: renderer = get_config().get('renderer', None)
    if renderer is None: renderer=BasicMarkdownRenderer
    elif isinstance(renderer,str):
        p,m = renderer.rsplit('.', 1)
        renderer = getattr(import_module(p), m)
    return renderer(sym or show_doc, disp=disp)

You can use `show_doc` to document apis of functions, classes or methods:

In [593]:
def f(x:int=1):
    "func docstring"
    ...

show_doc(f)

---

#### f

> **`f`**` (x: int = 1)`

func docstring

In [594]:
class Foo:
    def __init__(d:str,e:int):
        "This is the docstring for the __init__ method"
        ...

show_doc(Foo)

---

### Foo

> **`Foo`**` (e: int)`

This is the docstring for the __init__ method

In [595]:
class Foo:
    def a_method(a:list,b:dict,c):
        "This is a method"
        ...

show_doc(Foo.a_method)

---

#### Foo.a_method

> **`Foo.a_method`**` (a: list, b: dict, c)`

This is a method

In [25]:
#|export
class BasicHtmlRenderer(ShowDocRenderer):
    def _repr_html_(self):
        doc = '<hr/>\n'
        lvl = 4 if self.isfunc else 3
        doc += f'<h{lvl}>{self.nm}</h{lvl}>\n<blockquote><code>{self.nm}{self.sig}</code></blockquote>'
        if self.docs: doc += f"<p>{self.docs}</p>"
        return doc

In [26]:
class F:
    "class docstring"
    def __init__(self, x:int=1): ...

show_doc(F, renderer=BasicHtmlRenderer)

In [27]:
#|export
def showdoc_nm(tree):
    "Get the fully qualified name for showdoc."
    return ifnone(get_patch_name(tree), tree.name)

In [28]:
#|hide
import ast
code="""
@bar
@patch
@foo
def a_method(self:Foo, a:list,b:dict,c):
    "This is a method"
    ...
"""

code2="""
@bar
@foo
def a_method(self:Foo, a:list,b:dict,c):
    "This is a method"
    ...
"""

_tree = ast.parse(code).body[0]
test_eq(showdoc_nm(_tree), 'Foo.a_method')

_tree2 = ast.parse(code2).body[0]
test_eq(showdoc_nm(_tree2), 'a_method')

## Export -

In [29]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()